In [ ]:
import numpy as np
import pandas as pd

# Set seed of random number generator
CWID = -1 # Place here your Campus wide ID number, this will personalize
#your results, but still maintain the reproducibl nature of using seeds.
personal = CWID % 10000
np.random.seed(personal)

In [ ]:
# Import financial headlines
headlines_df = pd.read_csv('300_stock_headlines.csv')
headlines_df.head(5)

In [ ]:
# Save the headlines as a list
headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array) # Reshuffle headlines to avoid causal biases
headlines_list = list(headlines_array[:,2])

print(headlines_list)

In [ ]:
# Import FinBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
# Tokenize the headlines to be machine readable

inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
print(inputs) # Store machine readable headlines as "inputs"

In [ ]:
# Run FinBERT on the tokenized headlines

outputs = model(**inputs)
print(outputs.logits.shape) # Print out the outputted shape -- 3 outputs!

In [ ]:
#Reshape the outputs with softmax so that they sum to 1 and all are positive 
# This makes for easier interpretation of the output
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

In [ ]:
# Print the labels for each of the 3 outputs to understand how to interpret the results

model.config.id2label

In [ ]:
# Format the results as a pandas data frame
#Headline #Positive #Negative #Neutral

positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

df.head(5)
